In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import numpy as np
import os

In [2]:
# Ruta de la carpeta logs, que está un nivel arriba del notebook
logs_dir = os.path.join(os.getcwd(), '..', 'logs')   # Sube un nivel y entra a logs
logs_dir = os.path.abspath(logs_dir)                 # Convierte a ruta absoluta

In [3]:
print("Buscando en:", logs_dir)
print("¿Existe 'logs'? ->", os.path.isdir(logs_dir))

Buscando en: c:\Users\jdvil\Documents\Python\Practicas\Juego_Vida\logs
¿Existe 'logs'? -> True


In [ ]:
if os.path.isdir(logs_dir):
    csv_files = [f for f in os.listdir(logs_dir) if f.endswith('.csv')]
    print("Archivos CSV encontrados:", csv_files)
    if csv_files:
        csv_files.sort(key=lambda f: os.path.getmtime(os.path.join(logs_dir, f)), reverse=True)
        csv_path = os.path.join(logs_dir, csv_files[0])
        print("Cargando archivo:", csv_files[0])
        df = pd.read_csv(csv_path)
        print(df.head())
    else:
        print("No hay archivos CSV en la carpeta logs.")
else:
    print("La carpeta 'logs' no existe en la ubicación esperada.")

Archivos CSV encontrados: ['civilizaciones_log_2025-06-23_175415.csv']
Cargando archivo: civilizaciones_log_2025-06-23_175415.csv
   ciclo civilizacion personalidad estrategia  fuerza  tecnologia  religion  \
0      1         Civ1  militarista       hawk      45         0.0       0.0   
1      1         Civ2       halcon       dove      45         0.0       0.0   
2      1         Civ3  equilibrado       hawk      45         0.0       0.0   
3      2         Civ1  militarista       hawk      70         1.2       0.0   
4      2         Civ2       halcon       dove      77         0.0       0.0   

   militar  complejidad  celdas  plantasTec  iglesias  cuarteles  
0      0.0          9.8      30           0         0          0  
1      0.0         10.2      30           0         0          0  
2      0.0          5.8      30           0         0          0  
3      0.0         11.6      45           1         0          0  
4      0.0          7.4      51           0         0       

In [ ]:
# Elige solo las columnas numéricas relevantes para correlación
cols_corr = [
    'fuerza', 'tecnologia', 'religion', 'militar', 'complejidad',
    'celdas', 'plantasTec', 'iglesias', 'cuarteles'
]

cols_exist = [col for col in cols_corr if col in df.columns]
corr = df[cols_exist].corr()

# Gráfico de mapa de calor
plt.figure(figsize=(9,7))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Matriz de correlación entre variables')
plt.tight_layout()
plt.savefig('kpi_imgs/matriz_correlacion.png')
plt.close()

print("¡Matriz de correlación generada en kpi_imgs/advanced_correlacion.png!")


¡Matriz de correlación generada en kpi_imgs/advanced_correlacion.png!


In [11]:
# ========= 1. REGRESIÓN LINEAL: FUERZA vs. CELDAS (por civilización) =========
plt.figure(figsize=(8,6))
for civ in df['civilizacion'].unique():
    sub = df[df['civilizacion'] == civ]
    X = sub[['celdas']]
    y = sub['fuerza']
    model = LinearRegression().fit(X, y)
    pred = model.predict(X)
    plt.scatter(X, y, label=f'{civ}', s=18)
    plt.plot(X, pred, '--')
plt.title('Regresión Lineal: Fuerza vs Celdas')
plt.xlabel('Celdas')
plt.ylabel('Fuerza')
plt.legend()
plt.tight_layout()
plt.savefig('kpi_imgs/reg_fuerza_vs_celdas.png')
plt.close()

In [12]:
# ========= 2. REGRESIÓN LINEAL: COMPLEJIDAD vs. NÚMERO DE EDIFICIOS =========
df['total_edificios'] = df['plantasTec'] + df['iglesias'] + df['cuarteles']
plt.figure(figsize=(8,6))
for civ in df['civilizacion'].unique():
    sub = df[df['civilizacion'] == civ]
    X = sub[['total_edificios']]
    y = sub['complejidad']
    model = LinearRegression().fit(X, y)
    pred = model.predict(X)
    plt.scatter(X, y, label=f'{civ}', s=18)
    plt.plot(X, pred, '--')
plt.title('Regresión Lineal: Complejidad vs Total de Edificios')
plt.xlabel('Total de Edificios')
plt.ylabel('Complejidad')
plt.legend()
plt.tight_layout()
plt.savefig('kpi_imgs/reg_complejidad_vs_edificios.png')
plt.close()

In [13]:
# ========= 3. REGRESIÓN MÚLTIPLE: FUERZA FINAL vs. TODAS LAS MÉTRICAS =========
finals = df[df['ciclo'] == df['ciclo'].max()]
X = finals[['celdas','tecnologia','religion','militar','complejidad','plantasTec','iglesias','cuarteles']]
y = finals['fuerza']
model = LinearRegression().fit(X, y)
coefs = list(zip(X.columns, model.coef_))

# Guarda los coeficientes como imagen
fig, ax = plt.subplots(figsize=(7,3))
coef_names = [c for c, _ in coefs]
coef_vals = [v for _, v in coefs]
bars = ax.barh(coef_names, coef_vals, color='deepskyblue')
ax.set_xlabel('Peso')
ax.set_title('Coeficientes regresión múltiple (fuerza final)')
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2,
            f'{width:.3f}', va='center', ha='left', fontsize=10)
plt.tight_layout()
plt.savefig('kpi_imgs/reg_multiple_fuerza_final.png')
plt.close()

In [15]:
# ========= 5. CLUSTERIZACIÓN: K-MEANS POR PATRÓN DE CRECIMIENTO =========
# Vamos a agrupar por fuerza y celdas, solo en el ciclo final
X_clust = finals[['fuerza', 'celdas']]
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_clust)
finals['cluster'] = kmeans.labels_
plt.figure(figsize=(7,6))
colors = ['teal','orange','crimson','mediumorchid','gold','skyblue','lime','brown','pink','gray']
for i, c in enumerate(finals['cluster'].unique()):
    sub = finals[finals['cluster']==c]
    plt.scatter(sub['celdas'], sub['fuerza'], color=colors[i], s=100, label=f'Cluster {c+1}')
for idx, row in finals.iterrows():
    plt.text(row['celdas'], row['fuerza'], row['civilizacion'], fontsize=10, ha='center', va='bottom', color='white')
plt.xlabel('Celdas')
plt.ylabel('Fuerza')
plt.title('Clusters de civilizaciones por patrón final')
plt.legend()
plt.tight_layout()
plt.savefig('kpi_imgs/clusters_civilizaciones.png')
plt.close()

C:\Users\jdvil\AppData\Local\Temp\ipykernel_10960\2608859053.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finals['cluster'] = kmeans.labels_


In [16]:
# ========= 6. DETECCIÓN DE OUTLIERS: SALTOS ANÓMALOS EN FUERZA Y COMPLEJIDAD =========
df['delta_fuerza'] = df.groupby('civilizacion')['fuerza'].diff().fillna(0)
df['delta_complejidad'] = df.groupby('civilizacion')['complejidad'].diff().fillna(0)
plt.figure(figsize=(10,5))
for civ in df['civilizacion'].unique():
    sub = df[df['civilizacion']==civ]
    plt.plot(sub['ciclo'], sub['delta_fuerza'], label=f'{civ} ΔFuerza')
    plt.plot(sub['ciclo'], sub['delta_complejidad'], '--', label=f'{civ} ΔComplejidad')
plt.title("Saltos anómalos (fuerza y complejidad por ciclo)")
plt.xlabel("Ciclo")
plt.ylabel("Diferencia respecto al ciclo anterior")
plt.legend()
plt.tight_layout()
plt.savefig('kpi_imgs/outliers_saltos_fuerza_complejidad.png')
plt.close()

In [17]:
# ========= 7. ANÁLISIS DE TENDENCIA TEMPORAL: ACELERACIÓN/DECELERACIÓN EN CRECIMIENTO =========
plt.figure(figsize=(10,5))
for civ in df['civilizacion'].unique():
    sub = df[df['civilizacion']==civ]
    # 2da derivada: aceleración (aproximada)
    aceleracion = np.gradient(np.gradient(sub['fuerza']))
    plt.plot(sub['ciclo'], aceleracion, label=civ)
plt.title('Aceleración/deceleración del crecimiento de fuerza')
plt.xlabel('Ciclo')
plt.ylabel('Aceleración')
plt.legend()
plt.tight_layout()
plt.savefig('kpi_imgs/tendencia_aceleracion_fuerza.png')
plt.close()